<a href="https://colab.research.google.com/github/pejmanS21/deep-utils-notebooks/blob/main/Deep_Utils_Tutorial_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deep_utils
### Tutorial pt.2

## Download deep_utils library

In [ ]:
!pip install -U deep_utils

     |████████████████████████████████| 248 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## download image from web for testing

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/en/a/a0/Grogu_%28Star_Wars%29.jpg

--2022-03-30 12:44:43--  https://upload.wikimedia.org/wikipedia/en/a/a0/Grogu_%28Star_Wars%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33307 (33K) [image/jpeg]
Saving to: ‘Grogu_(Star_Wars).jpg’

Grogu_(Star_Wars).j 100%[===================>]  32.53K  --.-KB/s    in 0.05s   

2022-03-30 12:44:44 (679 KB/s) - ‘Grogu_(Star_Wars).jpg’ saved [33307/33307]



## Tutorial pt.2

### import necessary libraries

In [ ]:
import torch
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### BlocksTorch
easy way to create deep learning models for PyTorch 
<br> like Convolution blocks, Normalization, Activation and ...
```python
from deep_utils import BlocksTorch
```

In [ ]:
from deep_utils import BlocksTorch

#### Convolution layer

In [ ]:
conv = BlocksTorch.conv_norm_act(3, 32, k=(3, 3), s=(1, 1), p=(1, 1),
                      norm='bn', act='relu', conv=True, index=0,
                      pooling='max', pooling_k=(2, 2), pooling_s=(2, 2), move_forward=0)

print(conv)

Sequential(
  (conv2d_0_in3_f32_k3_s1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2d_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_0): ReLU()
  (max2d_0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)


#### input and output

In [ ]:
x = torch.randn(1, 3, 256, 256)

y = conv(x)

print(x.size(), y.size())

torch.Size([1, 3, 256, 256]) torch.Size([1, 32, 128, 128])


#### pooling layer

In [ ]:
pool_layer = BlocksTorch.load_pooling('avg', k=(2, 2), s=(2, 2))

print(pool_layer)

AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)


#### input and output

In [ ]:
y = pool_layer(x)

print(x.size(), y.size())

torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 128, 128])


#### activation layer

In [ ]:
act_layer = BlocksTorch.load_activation(act='leaky_relu')

print(act_layer)

LeakyReLU(negative_slope=0.01)


#### input and output

In [ ]:
y = act_layer(x)

print(x.size(), y.size())

torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 256, 256])


#### Normalization layer

In [ ]:
norm_layer = BlocksTorch.load_layer_norm(norm='bn', out_c=3)

print(norm_layer)

BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


#### input and output

In [ ]:
y = norm_layer(x)

print(x.size(), y.size())

torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 256, 256])


In [ ]:
res_block = BlocksTorch.res_basic_block(3, 3, False, down_sample=False)

print(res_block)

Sequential(
  (cnn_0_in3_f3): Sequential(
    (conv2d_0_in3_f3_k3_s1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2d_0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu_0): ReLU()
  )
  (cnn_1_in3_f3): Sequential(
    (conv2d_0_in3_f3_k3_s1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2d_0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


#### input and output

In [ ]:
y = res_block(x)

print(x.size(), y.size())

torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 256, 256])


### BlocksTF
easy way to create deep learning models for TensorFlow 
<br> like Convolution blocks, Normalization, Activation and ...
```python
from deep_utils import BlocksTF
```


In [ ]:
from deep_utils import BlocksTF

In [ ]:
conv = BlocksTF.conv_norm_act(64, k=(3, 3), s=(1, 1), p='same',
                      norm='bn', act='relu', conv=True, index=0,
                      pooling='avg', pooling_k=(2, 2), pooling_s=(2, 2),
                      pool_kwargs=None, move_forward=0)

conv.build((None, 256, 256, 3))
print(conv.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 64)     256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256, 256, 64)      0         
                                                                 
 average_pooling2d (AverageP  (None, 128, 128, 64)     0         
 ooling2D)                                                       
                                                                 
Total params: 2,048
Trainable params: 1,920
Non-trainable params: 128
_________________________________________________________________
None
